In [ ]:
import pandas as pd
import json
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

In [ ]:
############# Preprocessing sets
cancer = 'BRCA'
#############

# Correlation Analysis

In [ ]:

############# Correlation sets
# Options 'gnosis:BRCA:1', 
#         'BRCA:CF|All_Top 100',
#         'AKLIMATE_BRCA_reduced_model_1000_feature_set',
#         'nn_jg_2020-03-20_top1kfreq:BRCA', 
#         'fbedeBIC_BRCA'
grp1 = 'gnosis:BRCA:1'
grp2 = 'BRCA:CF|All_Top 100'
cancer = 'BRCA'
#############

In [ ]:
ft_df = pd.read_csv('../data/correlation/CORR_' + cancer + '_featurelist.tsv', sep = '\t')

In [ ]:
master_df = pd.read_csv('../src/tarball/' + cancer + '_v8_20200203.tsv', sep='\t', index_col=0)
# master_df

How much overlap?

In [ ]:
venn2([set(ft_df[grp1].dropna()), set(ft_df[grp2].dropna())], set_labels=[grp1,grp2])
plt.show()

Of those not overlapping, are they correlated?

In [ ]:
# Get feature lists
grp1_fts = ft_df[grp1].dropna()
grp2_fts = ft_df[grp2].dropna()

# Dataframe prep
cols = ['corr', 'pval', 'gene1', 'gene2', 'group1', 'group2']
gene1 = []
gene2 = []
corr = []
pval = []
group1 = []
group2 = []
    
# calculate corr
for g1 in grp1_fts:
    for g2 in grp2_fts:
#         if 'GEXP' in g1 and 'GEXP' in g2:
        if g1 == g2:
            pass
        else:
            # correlation matrix (coor coefficients)
            x = master_df[g1]
            y = master_df[g2]
            c, p = scipy.stats.spearmanr(x, y)

            gene1.append(g1)
            gene2.append(g2)
            corr.append(c)
            pval.append(p)
            group1.append(grp1)
            group2.append(grp2)

# save results
res = pd.DataFrame(list(zip( corr, pval,gene1, gene2, group1, group2)), columns = cols)
res = res.reindex(res['corr'].abs().sort_values(ascending=False).index).reset_index(drop=True) # sort by abs corr
res.to_csv('../data/correlation/out/corr_matrix_' + grp1 + '_' + grp2 + '.tsv', sep='\t',index=False)
res

In [ ]:
n_correlated = len(res[res['corr']> 0.5]) + len(res[res['corr']< -0.5])
print(n_correlated, 'correlated genes pairs (', (n_correlated/res.shape[0] )*100, '%) between ', grp1, grp2)
print('excludes instances where both groups picked same gene')

##### Log (stored in `data/correlation/out/` with prefix `corr_matrix_`)

Between JADBIO and CloudForest (gnosis:BRCA:1, BRCA:CF|All_Top 100)
+ 11 overlaps
+ 560 correlated genes pairs ( 22.49899558055444 %) excludes instances where both groups picked same gene. 

Between JADBIO and AKLIMATE (gnosis:BRCA:1, AKLIMATE_BRCA_reduced_model_1000_feature_set)
+ 22 overlaps
+ 5295 correlated genes pairs ( 7.908532851403223 %) excludes instances where both groups picked same gene


Between JADBIO and SubSCOPE (gnosis:BRCA:1, nn_jg_2020-03-20_top1kfreq:BRCA)
+ 2 overlaps
+ 432 correlated genes pairs ( 2.179286687181557 %) excludes instances where both groups picked same gene


Between JADBIO and SciKit_grid
+ 
+ 



Between CloudForest and AKLIMATE (AKLIMATE_BRCA_reduced_model_1000_feature_set, BRCA:CF|All_Top 100)
+ 96 overlaps
+ 19652 correlated genes pairs ( 7.338202566055772 %) excludes instances where both groups picked same gene

Between CloudForest and SubSCOPE (BRCA:CF|All_Top 100, nn_jg_2020-03-20_top1kfreq:BRCA)
+ 5 overlaps
+ 1550 correlated genes pairs ( 1.954726023078378 %) excludes instances where both groups picked same gene

Between CloudForest and SciKit_grid
+ 
+ 

Between AKLIMATE and SubSCOPE
+ 239 overlaps
+ 

Between AKLIMATE and SciKit_grid
+ 
+ 

Between SubSCOPE and SciKit_grid
+ 
+ 

# Suggestions

In [ ]:
# then is this number more or less than we expect randomly -- we can do a null model
# what if those 2 groups picked randomly any genes (from orig matix) > until fill
# then repeat 100 times (check for more correlations, but could do 2 tailed )
# ---eventually play with pval thres
# then plot distribut of this null model

In [ ]:
# explore miRNA
res_dict = {}

for col in ft_df.columns:
    vals = []
    for f in ft_df[col].dropna():
        if 'N:MIR' in f:
            vals.append(f)
    res_dict[col]=vals


In [ ]:
mi_df = pd.DataFrame.from_dict(res_dict,orient='index').T
print('these are the miRNA features in the best performing models')
mi_df